<h1><center>PyDyNo: Analysis of signal execution modes in biochemical networks</center></h1>
<h1><center>Supplemental Material</center></h1>

<font size="4">This tutorial for PyDyNo will go through the analysis pipeline for analyzing the signal execution modes in a biochemical network. First, import the necessary packages to run the analysis, including importing your model.</font>

In [ ]:
from necro_uncal_new_updated import model
import pydyno
from pylab import *
from pysb.core import *
from pysb.bng import *
from pysb.integrate import *
import matplotlib.pyplot as plt
import numpy as np
from pysb.util import alias_model_components
from pysb.simulator import ScipyOdeSimulator
import pandas as pd
from scipy import stats
from pysb.simulator import SimulationResult
from pydyno.discretization.pysb_discretize import PysbDomPath
from pydyno.seqanalysis import SeqAnalysis
from pydyno.visualize_simulations import VisualizeSimulations
from pydyno.visualize_discretization import visualization_path
from pydyno.visualize_discretization import visualization_seq_paths
import random
import h5py
%matplotlib inline

<font size="4">Following model calibration to obtain the fitted parameter sets for the model, the next step is to use the model and simulations from fitted parameter sets to identify the dominant paths and signatures in the network.</font>

<font size="4">For PySB models or SBML models, the `PysbDomPath` or `SbmlDomPath` functions will be used respectively to identify these dominant paths, and then obtaining signatures, and paths associated with the given model and simulation results. These results will discretize the simulated fitted model trajectories of a specific model species. </font>

<font size="4">Example:</font>
```python
tspan = np.linspace(0, 20000, 100)

sim = ScipyOdeSimulator(model, tspan).run()

dp = PysbDomPath(model=model, simulations=sim) or SbmlDomPath(model=model, simulations=sim)

signatures, paths = dp.get_path_signatures(target='s37', type_analysis='consumption', depth=5, dom_om=1)
```
<font size="4">Optional: To save your signatures for easier reload at another time, signatures can be saved using h5 below: </font>
```python
signatures.save('signatures_10k_1om_15depth_seqs_100tnf.h5')
```

<font size="4">Below, you will specify your `tspan`, initiate your solver of choice, following the resulting `run` of your simulator. This tutorial will use `ScipyOdeSimulator`. Finally, save your result object as an `h5` file for future use. </font>

<font size="4">For reference loading your `h5` file: </font>
```python
sim10 = SimulationResult.load('necro_pydream_5chns_929_10tnf_updated_nokd.h5')
```
</font>

In [ ]:
tspan = np.linspace(0, 1440, 1440)
sim = ScipyOdeSimulator(model10, tspan=tspan, verbose = True)
result = sim.run(param_values=all_pars2[:], num_processors = 20)
result.save('necro_pydream_5chns_929_10tnf_updated_nokd_unique6707pars.h5')

<font size="4">Once you have obtained your simulation results from your fitted parameter sets, the next step is to import your `model` and simulation result object, `sim`, to the `PysbDomPath` function and obtain your `signatures` and `paths` information. </font>

<font size="4"> `dp.get_path_signatures` will require the `target`, your species index of interest, `type_analysis` which specify `consumption` or `production` of your `target` species. You will also specify the `depth`, of how many levels above your `target` node in the network you want to observe and analyze, and finally how many orders of magnitude `dom_om` in your network. </font>

<font size="4"> After obtaining the `signatures` and `paths`, you will take your signatures and calculate your `dissimilarity matrix` using `signatures.dissimilarity_matrix` </font>

<font size="4"> Optional to save your signatures for easy reload later, you can run: </font>

```python 
signatures.save('signatures_10k_1om_sequences_100tnf_10000_new_14clusters.h5')```


In [ ]:
dp = PysbDomPath(model10, sim)
signatures, paths = dp.get_path_signatures(target='s36', type_analysis='production', 
                                             depth=15, dom_om=0.5, num_processors =20)
signatures.dissimilarity_matrix(n_jobs=100)

In [ ]:
signatures.save('signatures_10k_1om_sequences_100tnf_10000_new_14clusters.h5')

# Seqanalysis Modules 

In [ ]:
# random.seed(3005)
signatures.silhouette_score_spectral_range(range(2,20), n_jobs=100)

In [ ]:
signatures.spectral_clustering(2)

In [ ]:
vt = VisualizeSimulations(model100, sim_results='necro_pydream_5chns_929_100tnf_updated_nokd.h5', 
                          clusters=signatures.labels)

In [ ]:
vt.plot_cluster_dynamics(components=['MLKLa_obs'])

In [ ]:
# random.seed(3005)
plt.figure(figsize = (20,30))
signatures.plot_sequences(type_fig='modal')
# plt.savefig('pydyno_necro100tnf_s36obs_pmlkl_production.pdf', format='pdf')
plt.show()

In [ ]:
pmlkl = model100.monomers['MLKL']
vt.plot_pattern_sps_distribution(pattern=pmlkl(state='active'), type_fig='bar')
plt.show()

In [ ]:
paths.keys()

In [ ]:
print('cluster 0')
# print(signatures[1][32])
print(np.where(signatures.labels[:1000]==0))

In [ ]:
print('cluster 1')
# print(signatures[1][32])
print(np.where(signatures.labels[:1000]==0))

In [ ]:
print('cluster 0')
# print(signatures[1][32])
print(np.where(signatures.labels[:1000]==0))

In [ ]:
visualization_path(model, paths[135], type_analysis='production', 
                   filename='cluster0_arrestin_s36_production_1om_depth15_path135.png')
visualization_path(model, paths[10], type_analysis='production', 
                   filename='cluster0_arrestin_s36_production_1om_depth15_path10.png')
visualization_path(model, paths[173], type_analysis='production', 
                   filename='cluster0_arrestin_s36_production_1om_depth15_path173.png')
visualization_path(model, paths[167], type_analysis='production', 
                   filename='cluster0_arrestin_s36_production_1om_depth15_path167.png')

In [ ]:
visualization_seq_paths(sim, 0, signatures)

In [ ]:
cluster_label_idxs = VisualizeSimulations.check_clusters_arg(signatures.labels, nsims=10000)

In [ ]:
pars_cluster_0 = sims.param_values[cluster_label_idxs[0]]
print(pars_cluster_0)

In [ ]:
pars_cluster_0 = sims.param_values[cluster_labels_idxs[0]]